In [1]:
from pyspark.sql import HiveContext
sqlContext = HiveContext(sc)

In [5]:
sqlContext.sql("show tables").show(50)

+--------------------+-----------+
|           tableName|isTemporary|
+--------------------+-----------+
|        bike_seasons|      false|
|      bike_share_raw|      false|
|       complications|      false|
|      effective_care|      false|
|                 foo|      false|
|           hospitals|      false|
|     hospital_survey|      false|
|          infections|      false|
|            measures|      false|
|             my_care|      false|
|my_combined_proce...|      false|
|    my_complications|      false|
|        my_hospitals|      false|
|       my_infections|      false|
|     my_readmissions|      false|
|     my_star_ratings|      false|
|    my_survey_scores|      false|
|        readmissions|      false|
|             seasons|      false|
|            stadiums|      false|
|   surveys_responses|      false|
|           user_info|      false|
|             weather|      false|
|        weblogs_flat|      false|
|     weblogs_parquet|      false|
|      weblogs_schem

In [8]:
tables = ['complications', 'effective_care', 'hospitals', 'surveys_responses',
          'hospital_survey', 'infections', 'measures', 'readmissions',
          'my_complications', 'my_hospitals', 'my_infections', 'my_readmissions', 'my_star_ratings', 
          'my_survey_scores', 'my_combined_procedures', 'my_care']
for table in tables:
    sql = "DROP TABLE IF EXISTS %s" % (table)
    sqlContext.sql(sql)

In [10]:
sqlContext.sql("show tables").show(50)

+---------------+-----------+
|      tableName|isTemporary|
+---------------+-----------+
|   bike_seasons|      false|
| bike_share_raw|      false|
|            foo|      false|
|        seasons|      false|
|       stadiums|      false|
|      user_info|      false|
|        weather|      false|
|   weblogs_flat|      false|
|weblogs_parquet|      false|
| weblogs_schema|      false|
|web_session_log|      false|
+---------------+-----------+



In [62]:
sqlContext.sql("DROP TABLE my_hospitals")

sql = """
CREATE TABLE my_hospitals AS 
SELECT 
provider_id, hospital_name name, city, state, 
zip_code, hospital_type type, hospital_ownership ownership 
FROM hospitals
"""
sqlContext.sql(sql)

DataFrame[]

In [6]:
sqlContext.sql("show tables").show()

+-----------------+-----------+
|        tableName|isTemporary|
+-----------------+-----------+
|    complications|      false|
|   effective_care|      false|
|              foo|      false|
|        hospitals|      false|
|  hospital_survey|      false|
|       infections|      false|
|         measures|      false|
|     my_hospitals|      false|
|     readmissions|      false|
|         stadiums|      false|
|surveys_responses|      false|
|        user_info|      false|
|     weblogs_flat|      false|
|  weblogs_parquet|      false|
|   weblogs_schema|      false|
|  web_session_log|      false|
+-----------------+-----------+



In [8]:
sqlContext.sql("describe hospital_survey").show()

+--------------------+---------+-----------------+
|            col_name|data_type|          comment|
+--------------------+---------+-----------------+
|         provider_id|   string|from deserializer|
|       hospital_name|   string|from deserializer|
|             address|   string|from deserializer|
|                city|   string|from deserializer|
|               state|   string|from deserializer|
|            zip_code|   string|from deserializer|
|         county_name|   string|from deserializer|
|        phone_number|   string|from deserializer|
|   hcahps_measure_id|   string|from deserializer|
|     hcahps_question|   string|from deserializer|
|hcahps_answer_des...|   string|from deserializer|
|patient_survey_st...|   string|from deserializer|
|patient_survey_st...|   string|from deserializer|
|hcahps_answer_per...|   string|from deserializer|
|hcahps_answer_per...|   string|from deserializer|
|number_of_complet...|   string|from deserializer|
|number_of_complet...|   string

In [63]:
sql = """DROP TABLE my_star_ratings"""
sqlContext.sql(sql)

# create new table
sql = """
CREATE TABLE my_star_ratings AS 
SELECT 
provider_id, hcahps_measure_id measure_id, 
hcahps_question question, hcahps_answer_description answer,
cast(patient_survey_star_rating AS INT) star_rating,
cast(to_date(from_unixtime(unix_timestamp(measure_start_date, 'MM/dd/yy'))) as date) measure_start,
cast(to_date(from_unixtime(unix_timestamp(measure_end_date, 'MM/dd/yy'))) as date) measure_end
FROM hospital_survey
"""
sqlContext.sql(sql)

DataFrame[]

In [65]:
sqlContext.sql('describe my_star_ratings').show()

+-------------+---------+-------+
|     col_name|data_type|comment|
+-------------+---------+-------+
|  provider_id|   string|   null|
|   measure_id|   string|   null|
|     question|   string|   null|
|       answer|   string|   null|
|  star_rating|      int|   null|
|measure_start|     date|   null|
|  measure_end|     date|   null|
+-------------+---------+-------+



In [66]:
sqlContext.sql('select * from my_star_ratings limit 10').show()

+-----------+--------------------+--------------------+--------------------+-----------+-------------+-----------+
|provider_id|          measure_id|            question|              answer|star_rating|measure_start|measure_end|
+-----------+--------------------+--------------------+--------------------+-----------+-------------+-----------+
|     010001|       H_STAR_RATING| Summary star rating| Summary star rating|          3|   2013-10-01| 2014-09-30|
|     010001|     H_CLEAN_HSP_A_P|Patients who repo...|Room was "always"...|       null|   2013-10-01| 2014-09-30|
|     010001|    H_CLEAN_HSP_SN_P|Patients who repo...|Room was "sometim...|       null|   2013-10-01| 2014-09-30|
|     010001|     H_CLEAN_HSP_U_P|Patients who repo...|Room was "usually...|       null|   2013-10-01| 2014-09-30|
|     010001| H_CLEAN_STAR_RATING|Cleanliness - sta...|Cleanliness - sta...|          2|   2013-10-01| 2014-09-30|
|     010001|        H_COMP_1_A_P|Patients who repo...|Nurses "always" c...|    

In [14]:
sql = """
CREATE TABLE my_survey_scores AS 
SELECT 
provider_number provider_id, 
cast(hcahps_base_score as INT) base_score,
cast(hcahps_consistency_score as INT) consistency_score,
cast(hcahps_base_score as INT) + cast(hcahps_consistency_score as INT) total_score
FROM surveys_responses
"""
sqlContext.sql(sql)

++
||
++
++



In [15]:
sqlContext.sql("select * from my_survey_scores limit 10").show()

+-----------+----------+-----------------+-----------+
|provider_id|base_score|consistency_score|total_score|
+-----------+----------+-----------------+-----------+
|     010001|        17|               16|         33|
|     010005|        51|               20|         71|
|     010006|         9|               12|         21|
|     010007|        40|               17|         57|
|     010008|        30|               15|         45|
|     010011|        11|               16|         27|
|     010012|        31|               20|         51|
|     010016|        10|               15|         25|
|     010019|        24|               20|         44|
|     010021|        34|               19|         53|
+-----------+----------+-----------------+-----------+



In [17]:
sqlContext.sql("describe my_survey_scores").show()

+-----------------+---------+-------+
|         col_name|data_type|comment|
+-----------------+---------+-------+
|      provider_id|   string|   null|
|       base_score|      int|   null|
|consistency_score|      int|   null|
|      total_score|      int|   null|
+-----------------+---------+-------+



In [21]:
sql = """
DROP TABLE my_care
"""
sqlContext.sql(sql)


sql = """
CREATE TABLE my_care AS 
SELECT 
provider_id, condition, measure_id,
measure_name,
cast(score as INT) score,
100 - cast(score as DECIMAL(16,4)) new_score,
cast(to_date(from_unixtime(unix_timestamp(measure_start_date, 'MM/dd/yy'))) as date) measure_start,
cast(to_date(from_unixtime(unix_timestamp(measure_end_date, 'MM/dd/yy'))) as date) measure_end
FROM effective_care
WHERE
measure_id = 'OP_22' OR measure_id = 'PC_01' OR measure_id = 'VTE_6'
"""
sqlContext.sql(sql)



DataFrame[]

In [23]:
sql = """
INSERT INTO TABLE my_care
SELECT 
provider_id, condition, measure_id,
measure_name,
cast(score as INT) score,
cast(score as DECIMAL(16,4)) new_score,
cast(to_date(from_unixtime(unix_timestamp(measure_start_date, 'MM/dd/yy'))) as date) measure_start,
cast(to_date(from_unixtime(unix_timestamp(measure_end_date, 'MM/dd/yy'))) as date) measure_end
FROM effective_care
WHERE
measure_id != 'OP_22' AND measure_id != 'PC_01' AND measure_id != 'VTE_6'
"""
sqlContext.sql(sql)

DataFrame[]

In [26]:
sqlContext.sql("describe my_care").show()

+-------------+-------------+-------+
|     col_name|    data_type|comment|
+-------------+-------------+-------+
|  provider_id|       string|   null|
|    condition|       string|   null|
|   measure_id|       string|   null|
| measure_name|       string|   null|
|        score|          int|   null|
|    new_score|decimal(17,4)|   null|
|measure_start|         date|   null|
|  measure_end|         date|   null|
+-------------+-------------+-------+



In [31]:
sqlContext.sql("select * from my_care WHERE measure_id == 'AMI_10'").show()

+-----------+--------------------+----------+-------------------+-----+---------+-------------+-----------+
|provider_id|           condition|measure_id|       measure_name|score|new_score|measure_start|measure_end|
+-----------+--------------------+----------+-------------------+-----+---------+-------------+-----------+
|     010001|Heart Attack or C...|    AMI_10|Statin at Discharge|   98|  98.0000|   2013-10-01| 2014-09-30|
|     010005|Heart Attack or C...|    AMI_10|Statin at Discharge|  100| 100.0000|   2013-10-01| 2014-09-30|
|     010006|Heart Attack or C...|    AMI_10|Statin at Discharge|  100| 100.0000|   2013-10-01| 2014-09-30|
|     010007|Heart Attack or C...|    AMI_10|Statin at Discharge| null|     null|   2013-10-01| 2014-09-30|
|     010008|Heart Attack or C...|    AMI_10|Statin at Discharge| null|     null|   2013-10-01| 2014-09-30|
|     010011|Heart Attack or C...|    AMI_10|Statin at Discharge|  100| 100.0000|   2013-10-01| 2014-09-30|
|     010012|Heart Attack or

In [47]:
sql = """
DROP TABLE my_infections
"""
sqlContext.sql(sql)


sql = """
CREATE TABLE my_infections AS 
SELECT 
provider_id, measure_id,
measure_name, compared_to_national,
cast(score as DECIMAL(16,4)) score,
cast(to_date(from_unixtime(unix_timestamp(measure_start_date, 'MM/dd/yy'))) as date) measure_start,
cast(to_date(from_unixtime(unix_timestamp(measure_end_date, 'MM/dd/yy'))) as date) measure_end
FROM infections
"""
sqlContext.sql(sql)

DataFrame[]

In [48]:
sqlContext.sql("describe my_infections").show()

+--------------------+-------------+-------+
|            col_name|    data_type|comment|
+--------------------+-------------+-------+
|         provider_id|       string|   null|
|          measure_id|       string|   null|
|        measure_name|       string|   null|
|compared_to_national|       string|   null|
|               score|decimal(16,4)|   null|
|       measure_start|         date|   null|
|         measure_end|         date|   null|
+--------------------+-------------+-------+



In [53]:
sqlContext.sql("select * from my_infections").show()

+-----------+---------------+--------------------+--------------------+---------+-------------+-----------+
|provider_id|     measure_id|        measure_name|compared_to_national|    score|measure_start|measure_end|
+-----------+---------------+--------------------+--------------------+---------+-------------+-----------+
|     010001| HAI_1_CI_LOWER|CLABSI Lower Conf...|No Different than...|   0.7370|   2013-10-01| 2014-09-30|
|     010001| HAI_1_CI_UPPER|CLABSI Upper Conf...|No Different than...|   3.7820|   2013-10-01| 2014-09-30|
|     010001|HAI_1_DOPC_DAYS|CLABSI Central Li...|No Different than...|2199.0000|   2013-10-01| 2014-09-30|
|     010001|HAI_1_ELIGCASES|CLABSI Predicted ...|No Different than...|   3.3000|   2013-10-01| 2014-09-30|
|     010001|HAI_1_NUMERATOR|CLABSI Observed C...|No Different than...|   6.0000|   2013-10-01| 2014-09-30|
|     010001|      HAI_1_SIR|Central line-asso...|No Different than...|   1.8180|   2013-10-01| 2014-09-30|
|     010001| HAI_2_CI_LOWER

In [54]:
sql = """
DROP TABLE my_readmissions
"""
sqlContext.sql(sql)


sql = """
CREATE TABLE my_readmissions AS 
SELECT 
provider_id, measure_id,
measure_name, compared_to_national,
cast(score as DECIMAL(16,4)) score,
cast(to_date(from_unixtime(unix_timestamp(measure_start_date, 'MM/dd/yy'))) as date) measure_start,
cast(to_date(from_unixtime(unix_timestamp(measure_end_date, 'MM/dd/yy'))) as date) measure_end
FROM readmissions
"""
sqlContext.sql(sql)

DataFrame[]

In [55]:
sqlContext.sql("describe my_readmissions").show()

+--------------------+-------------+-------+
|            col_name|    data_type|comment|
+--------------------+-------------+-------+
|         provider_id|       string|   null|
|          measure_id|       string|   null|
|        measure_name|       string|   null|
|compared_to_national|       string|   null|
|               score|decimal(16,4)|   null|
|       measure_start|         date|   null|
|         measure_end|         date|   null|
+--------------------+-------------+-------+



In [56]:
sqlContext.sql("select * from my_readmissions").show()

+-----------+------------------+--------------------+--------------------+-------+-------------+-----------+
|provider_id|        measure_id|        measure_name|compared_to_national|  score|measure_start|measure_end|
+-----------+------------------+--------------------+--------------------+-------+-------------+-----------+
|     010001|       MORT_30_AMI|Acute Myocardial ...|No different than...|12.2000|   2011-07-01| 2014-06-30|
|     010001|      MORT_30_CABG| Death rate for CABG|No different than...| 3.7000|   2011-07-01| 2014-06-30|
|     010001|      MORT_30_COPD|Death rate for ch...|No different than...| 8.8000|   2011-07-01| 2014-06-30|
|     010001|        MORT_30_HF|Heart failure (HF...|No different than...|12.6000|   2011-07-01| 2014-06-30|
|     010001|        MORT_30_PN|Pneumonia (PN) 30...|No different than...|11.6000|   2011-07-01| 2014-06-30|
|     010001|       MORT_30_STK|Death rate for st...|No different than...|15.3000|   2011-07-01| 2014-06-30|
|     010001|      

In [57]:
sql = """
DROP TABLE my_complications
"""
sqlContext.sql(sql)


sql = """
CREATE TABLE my_complications AS 
SELECT 
provider_id, measure_id,
measure_name, compared_to_national,
cast(score as DECIMAL(16,4)) score,
cast(to_date(from_unixtime(unix_timestamp(measure_start_date, 'MM/dd/yy'))) as date) measure_start,
cast(to_date(from_unixtime(unix_timestamp(measure_end_date, 'MM/dd/yy'))) as date) measure_end
FROM complications
"""
sqlContext.sql(sql)

DataFrame[]

In [59]:
sqlContext.sql("describe my_complications").show()

+--------------------+-------------+-------+
|            col_name|    data_type|comment|
+--------------------+-------------+-------+
|         provider_id|       string|   null|
|          measure_id|       string|   null|
|        measure_name|       string|   null|
|compared_to_national|       string|   null|
|               score|decimal(16,4)|   null|
|       measure_start|         date|   null|
|         measure_end|         date|   null|
+--------------------+-------------+-------+



In [60]:
sqlContext.sql("select * from my_complications").show()

+-----------+--------------------+--------------------+--------------------+--------+-------------+-----------+
|provider_id|          measure_id|        measure_name|compared_to_national|   score|measure_start|measure_end|
+-----------+--------------------+--------------------+--------------------+--------+-------------+-----------+
|     010001|       COMP_HIP_KNEE|Rate of complicat...|No different than...|  3.9000|   2011-04-01| 2014-03-31|
|     010001|PSI_12_POSTOP_PUL...|Serious blood clo...|Better than the N...|  2.1600|   2012-07-01| 2014-06-30|
|     010001| PSI_14_POSTOP_DEHIS|A wound that spli...|No different than...|  1.6700|   2012-07-01| 2014-06-30|
|     010001|      PSI_15_ACC_LAC|Accidental cuts a...|No different than...|  1.5200|   2012-07-01| 2014-06-30|
|     010001|     PSI_4_SURG_COMP|Deaths among Pati...|No different than...|138.3000|   2012-07-01| 2014-06-30|
|     010001|       PSI_6_IAT_PTX|Collapsed lung du...|No different than...|  0.3100|   2012-07-01| 2014

In [61]:
sqlContext.sql("show tables").show()

+-----------------+-----------+
|        tableName|isTemporary|
+-----------------+-----------+
|    complications|      false|
|   effective_care|      false|
|              foo|      false|
|        hospitals|      false|
|  hospital_survey|      false|
|       infections|      false|
|         measures|      false|
|          my_care|      false|
| my_complications|      false|
|     my_hospitals|      false|
|    my_infections|      false|
|  my_readmissions|      false|
|   my_star_rating|      false|
| my_survey_scores|      false|
|     readmissions|      false|
|         stadiums|      false|
|surveys_responses|      false|
|        user_info|      false|
|     weblogs_flat|      false|
|  weblogs_parquet|      false|
+-----------------+-----------+
only showing top 20 rows



In [29]:
df = sqlContext.sql("show tables")
df.registerTempTable('temp_tables')
query = """SELECT * FROM temp_tables WHERE tableName LIKE '%my%'"""
sqlContext.sql(query).show()

+----------------+-----------+
|       tableName|isTemporary|
+----------------+-----------+
|       my_tables|       true|
|         my_care|      false|
|my_complications|      false|
|    my_hospitals|      false|
|   my_infections|      false|
| my_readmissions|      false|
| my_star_ratings|      false|
|my_survey_scores|      false|
+----------------+-----------+



In [28]:
sqlContext.sql("show tables").show()

+-----------------+-----------+
|        tableName|isTemporary|
+-----------------+-----------+
|        my_tables|       true|
|    complications|      false|
|   effective_care|      false|
|              foo|      false|
|        hospitals|      false|
|  hospital_survey|      false|
|       infections|      false|
|         measures|      false|
|          my_care|      false|
| my_complications|      false|
|     my_hospitals|      false|
|    my_infections|      false|
|  my_readmissions|      false|
|  my_star_ratings|      false|
| my_survey_scores|      false|
|     readmissions|      false|
|         stadiums|      false|
|surveys_responses|      false|
|        user_info|      false|
|     weblogs_flat|      false|
+-----------------+-----------+
only showing top 20 rows



## Transforming data frames

In [19]:
from pyspark.sql import functions as func

# ignore the values that are not rates
sql = """
select provider_id, measure_name,
measure_id, score
from my_care 
where measure_id != 'ED_1b' AND measure_id != 'ED_2b' 
AND measure_id != 'EDV' AND measure_id != 'OP_1'
AND measure_id != 'OP_18b' AND measure_id != 'OP_20' AND measure_id != 'OP_21' 
AND measure_id != 'OP_3b' AND measure_id != 'OP_5'
"""
care_df = sqlContext.sql(sql)
care_df.count()

179164

In [3]:
# add new column for transformed score
care_df = care_df.withColumn('new_score', care_df.score)

# convert measure_id = 'OP_22' OR measure_id = 'PC_01' OR measure_id = 'VTE_6' to 100 - new_score
care_df = care_df.withColumn('new_score', 
                         func.when(care_df.measure_id == 'OP_22', 100 - care_df.new_score). \
                         otherwise(care_df.new_score))
care_df = care_df.withColumn('new_score', 
                             func.when(care_df.measure_id == 'PC_01', 100 - care_df.new_score). \
                             otherwise(care_df.new_score))
care_df = care_df.withColumn('new_score', 
                             func.when(care_df.measure_id == 'VTE_6', 100 - care_df.new_score).
                             otherwise(care_df.new_score))
care_df.count()
care_df.show()

+-----------+--------------------+----------------+-----+---------+
|provider_id|        measure_name|      measure_id|score|new_score|
+-----------+--------------------+----------------+-----+---------+
|     010001| Statin at Discharge|          AMI_10|   98|       98|
|     010001|Aspirin prescribe...|           AMI_2|   99|       99|
|     010001|Fibrinolytic Ther...|          AMI_7a| null|     null|
|     010001|Primary PCI Recei...|          AMI_8a|   91|       91|
|     010001|Emergency departm...|             EDV| null|     null|
|     010001|Discharge instruc...|            HF_1|   91|       91|
|     010001|Evaluation of LVS...|            HF_2|  100|      100|
|     010001|ACEI or ARB for LVSD|            HF_3|   98|       98|
|     010001|Immunization for ...|           IMM_2|   94|       94|
|     010001|Healthcare worker...|IMM_3_FAC_ADHPCT|   56|       56|
|     010001|Fibrinolytic Ther...|            OP_2| null|     null|
|     010001|Left before being...|           OP_

In [4]:
care_df[care_df.measure_id == 'VTE_6'].show()

+-----------+--------------------+----------+-----+---------+
|provider_id|        measure_name|measure_id|score|new_score|
+-----------+--------------------+----------+-----+---------+
|     010001|Incidence of pote...|     VTE_6|    0|      100|
|     010005|Incidence of pote...|     VTE_6|   29|       71|
|     010006|Incidence of pote...|     VTE_6|    8|       92|
|     010007|Incidence of pote...|     VTE_6| null|     null|
|     010008|Incidence of pote...|     VTE_6| null|     null|
|     010011|Incidence of pote...|     VTE_6|   19|       81|
|     010012|Incidence of pote...|     VTE_6| null|     null|
|     010016|Incidence of pote...|     VTE_6|    0|      100|
|     010018|Incidence of pote...|     VTE_6| null|     null|
|     010019|Incidence of pote...|     VTE_6| null|     null|
|     010021|Incidence of pote...|     VTE_6| null|     null|
|     010022|Incidence of pote...|     VTE_6| null|     null|
|     010023|Incidence of pote...|     VTE_6|    0|      100|
|     01

In [5]:
# select data from readmissions and deaths
sql = """
select provider_id, measure_name,
measure_id, score
from my_readmissions
"""
read_df = sqlContext.sql(sql)
print(read_df.count())

# convert scores to new_score so that higher is better
read_df = read_df.withColumn('new_score', read_df.score)
read_df = read_df.withColumn('new_score', 100 - read_df.new_score)
read_df.show()

66990
+-----------+--------------------+------------------+-------+---------+
|provider_id|        measure_name|        measure_id|  score|new_score|
+-----------+--------------------+------------------+-------+---------+
|     010001|Acute Myocardial ...|       MORT_30_AMI|12.2000|  87.8000|
|     010001| Death rate for CABG|      MORT_30_CABG| 3.7000|  96.3000|
|     010001|Death rate for ch...|      MORT_30_COPD| 8.8000|  91.2000|
|     010001|Heart failure (HF...|        MORT_30_HF|12.6000|  87.4000|
|     010001|Pneumonia (PN) 30...|        MORT_30_PN|11.6000|  88.4000|
|     010001|Death rate for st...|       MORT_30_STK|15.3000|  84.7000|
|     010001|Acute Myocardial ...|      READM_30_AMI|17.4000|  82.6000|
|     010001|Rate of unplanned...|     READM_30_CABG|15.9000|  84.1000|
|     010001|Rate of unplanned...|     READM_30_COPD|22.0000|  78.0000|
|     010001|Heart failure (HF...|       READM_30_HF|21.0000|  79.0000|
|     010001|Rate of readmissi...| READM_30_HIP_KNEE| 5.20

In [6]:
# select data from complications (measure_id != PSI_4_SURG_COMP)
sql = """
select provider_id, measure_name,
measure_id, score
from my_complications
where measure_id != 'PSI_4_SURG_COMP'
"""
comp_df = sqlContext.sql(sql)
print(comp_df.count())

# convert scores to new_score so that higher is better
comp_df = comp_df.withColumn('new_score', comp_df.score)
comp_df = comp_df.withColumn('new_score', 100 - comp_df.new_score)
comp_df.sort('score', ascending=False).show()

28065
+-----------+--------------------+--------------------+-------+---------+
|provider_id|        measure_name|          measure_id|  score|new_score|
+-----------+--------------------+--------------------+-------+---------+
|     140010|Serious blood clo...|PSI_12_POSTOP_PUL...|15.2000|  84.8000|
|     370093|Serious blood clo...|PSI_12_POSTOP_PUL...|14.6700|  85.3300|
|     330160|Serious blood clo...|PSI_12_POSTOP_PUL...|13.9900|  86.0100|
|     170027|Serious blood clo...|PSI_12_POSTOP_PUL...|13.7400|  86.2600|
|     310076|Serious blood clo...|PSI_12_POSTOP_PUL...|12.5400|  87.4600|
|     110083|Serious blood clo...|PSI_12_POSTOP_PUL...|12.1700|  87.8300|
|     140150|Serious blood clo...|PSI_12_POSTOP_PUL...|12.1100|  87.8900|
|     260105|Serious blood clo...|PSI_12_POSTOP_PUL...|11.9200|  88.0800|
|     260032|Serious blood clo...|PSI_12_POSTOP_PUL...|11.7300|  88.2700|
|     110161|Serious blood clo...|PSI_12_POSTOP_PUL...|11.3700|  88.6300|
|     390110|Serious blood clo..

In [7]:
# select data from infections only use Standardized Infections Ratios (SIR) for calculations
sql = """
select provider_id, measure_name,
measure_id, score
from my_infections
where measure_id like '%SIR%'
"""
infect_df = sqlContext.sql(sql)
print(infect_df.count())

# convert scores to new_score so that higher is better
infect_df = infect_df.withColumn('new_score', infect_df.score)
infect_df = infect_df.withColumn('new_score', 100 - infect_df.new_score)
infect_df.sort('score', ascending=False).show()

27936
+-----------+--------------------+----------+------+---------+
|provider_id|        measure_name|measure_id| score|new_score|
+-----------+--------------------+----------+------+---------+
|     100181|Methicillin-resis...| HAI_5_SIR|9.9890|  90.0110|
|     390270|Surgical Site Inf...| HAI_4_SIR|8.2870|  91.7130|
|     330233|Methicillin-resis...| HAI_5_SIR|7.3070|  92.6930|
|     230297|Methicillin-resis...| HAI_5_SIR|7.0380|  92.9620|
|     050008|Catheter-Associat...| HAI_2_SIR|6.7800|  93.2200|
|     480001|Catheter-Associat...| HAI_2_SIR|5.9170|  94.0830|
|     430012|Surgical Site Inf...| HAI_3_SIR|5.6540|  94.3460|
|     140095|Central line-asso...| HAI_1_SIR|5.6040|  94.3960|
|     110128|Catheter-Associat...| HAI_2_SIR|5.3480|  94.6520|
|     190005|Surgical Site Inf...| HAI_4_SIR|5.2360|  94.7640|
|     450039|Surgical Site Inf...| HAI_4_SIR|5.1280|  94.8720|
|     160047|Surgical Site Inf...| HAI_3_SIR|4.8660|  95.1340|
|     050425|Catheter-Associat...| HAI_2_SIR|4.81

In [8]:
# combine the all the tables
combined_df = care_df.unionAll(read_df).unionAll(comp_df).unionAll(infect_df)
combined_df.count()

306834

In [9]:
# convert null values to 0s
combined_df = combined_df.fillna({'new_score': 0})

In [10]:
# group by hospital, avg new_scores and sum new_scores
#combined_df.groupBy('provider_id') \
#    .agg(func.avg('new_score').alias('avg_score'),
#         func.sum('new_score').alias('sum_score')) \
#    .sort('avg_score', ascending=False).show()

In [21]:
from pyspark.sql import DataFrameWriter

#combined_df.registerTempTable('temp_care')
sqlContext.sql('DROP TABLE IF EXISTS my_combined_procedures')
df_writer = DataFrameWriter(combined_df)
df_writer.saveAsTable('my_combined_procedures')

In [22]:
sqlContext.sql("describe my_combined_procedures").show()

+------------+-------------+-------+
|    col_name|    data_type|comment|
+------------+-------------+-------+
| provider_id|       string|       |
|measure_name|       string|       |
|  measure_id|       string|       |
|       score|decimal(16,4)|       |
|   new_score|decimal(17,4)|       |
+------------+-------------+-------+



In [11]:
sqlContext.sql("select * from my_combined_procedures order by score DESC").show()

+-----------+--------------------+-----------+--------+---------+
|provider_id|        measure_name| measure_id|   score|new_score|
+-----------+--------------------+-----------+--------+---------+
|     010005|Prophylactic anti...| SCIP_INF_1|100.0000| 100.0000|
|     010006|Primary PCI Recei...|     AMI_8a|100.0000| 100.0000|
|     010005|Surgery Patients ...|SCIP_INF_10|100.0000| 100.0000|
|     010001|Prophylactic Anti...|       OP_6|100.0000| 100.0000|
|     010005|Prophylactic anti...| SCIP_INF_3|100.0000| 100.0000|
|     010001|Surgery Patients ...|SCIP_INF_10|100.0000| 100.0000|
|     010005|Surgery Patients ...| SCIP_VTE_2|100.0000| 100.0000|
|     010001|Surgery Patients ...| SCIP_VTE_2|100.0000| 100.0000|
|     010005|Assessed for Reha...|     STK_10|100.0000| 100.0000|
|     010001|Discharged on Ant...|      STK_2|100.0000| 100.0000|
|     010005|Anticoagulation T...|      STK_3|100.0000| 100.0000|
|     010005|Aspirin prescribe...|      AMI_2|100.0000| 100.0000|
|     0100

## Investigating data

In [8]:
# What hospitals are models of high-quality care? 
# That is, which hospitals have the most consistently 
# high scores for a variety of procedures.
sql = """
SELECT p.provider_id, avg(new_score) AS avg_score, 
sum(new_score) AS sum_score,
stddev(new_score) AS std_dev,
name, city, state
FROM my_combined_procedures p INNER JOIN my_hospitals h 
ON p.provider_id = h.provider_id
GROUP BY p.provider_id, name, city, state
ORDER BY avg_score DESC 
LIMIT 10
"""
combined_scores_df = sqlContext.sql(sql)
combined_scores_df.show()

+-----------+-----------+---------+------------------+--------------------+------------------+-----+
|provider_id|  avg_score|sum_score|           std_dev|                name|              city|state|
+-----------+-----------+---------+------------------+--------------------+------------------+-----+
|     450388|91.52250000|6223.5300|20.674949430551997|  METHODIST HOSPITAL|       SAN ANTONIO|   TX|
|     030036|91.03030769|5916.9700| 20.81949413774214|CHANDLER REGIONAL...|          CHANDLER|   AZ|
|     450723|90.92936923|5910.4090|21.152496027706285|METHODIST CHARLTO...|            DALLAS|   TX|
|     050231|90.69501538|5895.1760|20.944048846311603|POMONA VALLEY HOS...|            POMONA|   CA|
|     050308|90.65812308|5892.7780|21.745274866761417|  EL CAMINO HOSPITAL|     MOUNTAIN VIEW|   CA|
|     260065|90.60870769|5889.5660| 20.79683189165938|MERCY HOSPITAL SP...|       SPRINGFIELD|   MO|
|     050570|90.52480000|5884.1120|20.972100787768206|FOUNTAIN VALLEY R...|   FOUNTAIN VALL

In [7]:
#sqlContext.sql("SELECT * FROM my_combined_procedures where provider_id = '450388'").show(n=450388)

In [9]:
# What states are models of high-quality care?
sql = """
SELECT state, avg(new_score) AS avg_score, 
sum(new_score) AS sum_score,
stddev(new_score) AS std_dev
FROM my_combined_procedures p INNER JOIN my_hospitals h 
ON p.provider_id = h.provider_id
GROUP BY state
ORDER BY avg_score DESC
LIMIT 10
"""
sqlContext.sql(sql).show()

+-----+-----------+-----------+------------------+
|state|  avg_score|  sum_score|           std_dev|
+-----+-----------+-----------+------------------+
|   NJ|80.15610979|335854.1000|34.342725071482654|
|   DE|80.06332134| 33386.4050| 34.50229093623751|
|   CT|73.83223116|145966.3210| 39.39658237877559|
|   FL|73.29968334|868747.8470| 39.96398023239047|
|   RI|73.28125202| 54374.6890| 39.28783979441245|
|   MA|69.39629851|283622.6720| 42.01992836228156|
|   DC|68.26902377| 37343.1560| 41.56831749952921|
|   VA|67.73025287|366420.6680|43.053910645623695|
|   NY|66.36262537|747707.7000| 42.96222685958356|
|   PA|66.20132765|722124.0820| 43.49033522646685|
+-----+-----------+-----------+------------------+



In [10]:
# Which procedures have the greatest variability between hospitals?
sql = """
SELECT measure_id, measure_name, avg(new_score) AS avg_score, 
sum(new_score) AS sum_score,
stddev(new_score) AS std_dev
FROM my_combined_procedures
GROUP BY measure_id, measure_name
ORDER BY std_dev DESC
LIMIT 10
"""
sqlContext.sql(sql).show()

+-------------------+--------------------+-----------+-----------+------------------+
|         measure_id|        measure_name|  avg_score|  sum_score|           std_dev|
+-------------------+--------------------+-----------+-----------+------------------+
|          HAI_2_SIR|Catheter-Associat...|48.30051912|224887.2170| 49.45932191841454|
|              AMI_2|Aspirin prescribe...|46.76739691|217749.0000|49.334403774660466|
|          HAI_1_SIR|Central line-asso...|42.76890163|199132.0060| 49.26597269849545|
|              VTE_4|Unfractionated he...|43.39819588|202062.0000| 49.17729021645856|
|          HAI_3_SIR|Surgical Site Inf...|43.31772380|201687.3220| 49.14184522159113|
|             AMI_10| Statin at Discharge|45.86168385|213532.0000| 48.86002194975948|
|              STK_2|Discharged on Ant...|56.60674399|263561.0000| 48.84487628252045|
|PSI_14_POSTOP_DEHIS|A wound that spli...|54.71919029|254772.5500|48.829550373292854|
|          HAI_5_SIR|Methicillin-resis...|40.52777857|

In [31]:
# Are average scores for hospital quality or procedural variability correlated with patient survey responses?
sql = """
SELECT p.provider_id, avg(new_score) AS avg_score, 
sum(new_score) AS sum_score,
stddev(new_score) AS std_dev,
name, city, state,
total_score AS survey_score
FROM my_combined_procedures p INNER JOIN my_survey_scores s 
ON p.provider_id = s.provider_id INNER JOIN my_hospitals h 
ON p.provider_id = h.provider_id
GROUP BY p.provider_id, name, city, state, total_score
"""
cor_df = sqlContext.sql(sql)
cor_df.show()

+-----------+-----------+---------+------------------+--------------------+----------------+-----+------------+
|provider_id|  avg_score|sum_score|           std_dev|                name|            city|state|survey_score|
+-----------+-----------+---------+------------------+--------------------+----------------+-----+------------+
|     110229|80.61803077|5240.1720| 33.58515318042929|PIEDMONT NEWNAN H...|          NEWNAN|   GA|          40|
|     330045|83.30846154|5415.0500|30.749485314555415| HUNTINGTON HOSPITAL|      HUNTINGTON|   NY|          35|
|     360211|84.16970769|5471.0310|28.398505313713574|TRINITY MEDICAL C...|    STEUBENVILLE|   OH|          41|
|     360027|88.30530769|5739.8450|23.587604562842277|AKRON GENERAL MED...|           AKRON|   OH|          32|
|     100022|84.71827692|5506.6880| 26.61613362519213|JACKSON MEMORIAL ...|           MIAMI|   FL|          12|
|     500015|81.26541538|5282.2520| 33.26611564381173|MULTICARE AUBURN ...|          AUBURN|   WA|      

In [32]:
#cor_df = combined_scores_df.join(survey_df, on='provider_id', how='inner')
#cor_df.show()

In [33]:
# Are average scores for hospital quality correlated with patient survey scores?
cor_df.stat.corr('avg_score', 'survey_score')

-0.37048241485883027

In [34]:
cor_df.stat.corr('std_dev', 'survey_score')

0.30409843205406223

In [20]:
# Is procedural variability correlated with patient survey responses?
sql = """
SELECT measure_id, measure_name, avg(new_score) AS avg_score, 
sum(new_score) AS sum_score,
stddev(new_score) AS std_dev,
avg(total_score) AS avg_survey_score
FROM my_combined_procedures p INNER JOIN my_survey_scores s
ON p.provider_id = s.provider_id
GROUP BY measure_id, measure_name
"""
score_df = sqlContext.sql(sql)

In [21]:
score_df.stat.corr('std_dev', 'avg_survey_score')

0.052372713236259236

In [29]:
# Is procedural variability correlated with patient survey responses?
# Look at procedure variablity for each hospital

sql = """
SELECT p.provider_id, name, city, state,
avg(new_score) AS avg_score, 
sum(new_score) AS sum_score,
stddev(new_score) AS std_dev,
total_score AS survey_score
FROM my_combined_procedures p INNER JOIN my_survey_scores s
ON p.provider_id = s.provider_id INNER JOIN my_hospitals h
ON p.provider_id = h.provider_id
GROUP BY p.provider_id, name, city, state, total_score
ORDER BY std_dev DESC
"""
df = sqlContext.sql(sql)
df.show()

+-----------+--------------------+-------------------+-----+-----------+---------+------------------+------------+
|provider_id|                name|               city|state|  avg_score|sum_score|           std_dev|survey_score|
+-----------+--------------------+-------------------+-----+-----------+---------+------------------+------------+
|     490143|RIVERSIDE DOCTORS...|       WILLIAMSBURG|   VA|49.72800000|3232.3200|49.080679394708405|          62|
|     450766|UT SOUTHWESTERN U...|             DALLAS|   TX|46.76441538|3039.6870| 49.05240542518466|          81|
|     450851|BAYLOR HEART AND ...|             DALLAS|   TX|46.39875385|3015.9190|48.731317581527755|          84|
|     240006|OLMSTED MEDICAL C...|          ROCHESTER|   MN|44.90315385|2918.7050|48.589780704376594|          65|
|     060049|YAMPA VALLEY MEDI...|  STEAMBOAT SPRINGS|   CO|46.09643077|2996.2680| 48.42102750410094|          83|
|     460057|PARK CITY MEDICAL...|          PARK CITY|   UT|42.04540000|2732.951

In [30]:
df.stat.corr('std_dev', 'survey_score')

0.3040984320540629


## SQL Exploration


In [48]:
# ignore the values with times
sql = """
select provider_id, avg(new_score) as agg_score from my_care 
where measure_id != 'ED_1b' AND measure_id != 'ED_2b' AND measure_id != 'OP_1'
AND measure_id != 'OP_18b' AND measure_id != 'OP_20' AND measure_id != 'OP_21' 
AND measure_id != 'OP_3b' AND measure_id != 'OP_5'
group by provider_id order by agg_score DESC
"""
sqlContext.sql(sql).show()

+-----------+------------+
|provider_id|   agg_score|
+-----------+------------+
|     511321|100.00000000|
|     281324|100.00000000|
|     111319|100.00000000|
|     461302|100.00000000|
|     141332|100.00000000|
|     210058|100.00000000|
|     490144|100.00000000|
|     141329|100.00000000|
|     531302|100.00000000|
|     261333|100.00000000|
|     201304|100.00000000|
|     351315|100.00000000|
|     141326|100.00000000|
|     06007F|100.00000000|
|     261329|100.00000000|
|     201306|100.00000000|
|     141328|100.00000000|
|     201307|100.00000000|
|     231313|100.00000000|
|     491303|100.00000000|
+-----------+------------+
only showing top 20 rows



In [53]:
sql = """
select * from effective_care 
where measure_id != 'ED_1b' AND measure_id != 'ED_2b' AND measure_id != 'OP_1'
AND measure_id != 'OP_18b' AND measure_id != 'OP_20' AND measure_id != 'OP_21' 
AND measure_id != 'OP_3b' AND measure_id != 'OP_5'
AND provider_id = '281324'
"""
sqlContext.sql(sql).show()

+-----------+--------------------+-------------+------+-----+--------+-----------+------------+--------------------+-----------+--------------------+-------------+-------------+--------------------+------------------+----------------+
|provider_id|       hospital_name|      address|  city|state|zip_code|county_name|phone_number|           condition| measure_id|        measure_name|        score|       sample|            footnote|measure_start_date|measure_end_date|
+-----------+--------------------+-------------+------+-----+--------+-----------+------------+--------------------+-----------+--------------------+-------------+-------------+--------------------+------------------+----------------+
|     281324|NEMAHA COUNTY HOS...|2022  13TH ST|AUBURN|   NE|   68305|     NEMAHA|  4022744366|Heart Attack or C...|     AMI_10| Statin at Discharge|Not Available|Not Available|5 - Results are n...|        10/01/2013|      09/30/2014|
|     281324|NEMAHA COUNTY HOS...|2022  13TH ST|AUBURN|   NE

In [52]:
sqlContext.sql("select * from my_hospitals where provider_id = '281324'").show()

+-----------+--------------------+------+-----+--------+--------------------+------------------+
|provider_id|                name|  city|state|zip_code|                type|         ownership|
+-----------+--------------------+------+-----+--------+--------------------+------------------+
|     281324|NEMAHA COUNTY HOS...|AUBURN|   NE|   68305|Critical Access H...|Government - Local|
+-----------+--------------------+------+-----+--------+--------------------+------------------+

